# 分布鲁棒优化

**作者：**  
Daniel Kuhn  
风险分析与优化主席，  
洛桑联邦理工大学，瑞士洛桑  
电子邮件：[daniel.kuhn@epfl.ch](mailto:daniel.kuhn@epfl.ch)

Soroosh Shafiee  
康奈尔大学，运营研究与信息工程学院，  
美国纽约伊萨卡  
电子邮件：[shafiee@cornell.edu](mailto:shafiee@cornell.edu)

Wolfram Wiesemann  
帝国理工学院商学院，  
英国伦敦帝国理工大学  
电子邮件：[ww@imperial.ac.uk](mailto:ww@imperial.ac.uk)

---

## 摘要

分布鲁棒优化（DRO）研究不确定性下的决策问题，其中支配不确定问题参数的概率分布本身也是不确定的。任何DRO模型的一个关键组成部分是其模糊集，即与任何可用的结构或统计信息一致的概率分布族。DRO寻找在模糊集中的最坏分布下表现最好的决策。这一最坏情况标准得到了心理学和神经科学研究的支持，这些研究表明，许多决策者对分布不确定性有较低的容忍度。DRO根植于统计学、运筹学和控制理论，近期研究揭示了它与机器学习中的正则化技术和对抗训练之间的深刻联系。本综述以统一且自成体系的方式呈现了该领域的关键发现。

---

## 目录

1. [引言](#引言)
2. [模糊集](#模糊集)
3. [模糊集的拓扑属性](#模糊集的拓扑属性)
4. [最坏期望问题的对偶理论](#最坏期望问题的对偶理论)
5. [最坏风险问题的对偶理论](#最坏风险问题的对偶理论)
6. [自然子问题的解析解](#自然子问题的解析解)
7. [自然子问题的有限凸重构](#自然子问题的有限凸重构)
8. [通过鲁棒化进行正则化](#通过鲁棒化进行正则化)
9. [DRO问题的数值求解方法](#dro问题的数值求解方法)
10. [统计保证](#统计保证)
11. [参考文献](#参考文献)


## 1. 引言 (Introduction)
传统上，数学优化研究的是以下形式的问题：

$$
\inf _{x \in \mathcal{X}} \ell(x),
$$

其中，决策变量 $x$ 从可行解集 $\mathcal{X} \subseteq \mathbb{R}^n$ 中选择，目的是最小化损失函数 $\ell: \mathbb{R}^n \rightarrow \overline{\mathbb{R}}$。数学优化起源于17世纪末艾萨克·牛顿、戈特弗里德·威廉·莱布尼茨、皮埃尔·德·费马等人对微积分的研究，至今已有丰富的历史，涉及许多数学家、经济学家、工程师和科学家的贡献。现代数学优化的诞生通常归功于乔治·丹茨格，他在1947年提出的单纯形法可以求解线性优化问题，其中损失函数 $\ell$ 是仿射的，且可行集 $\mathcal{X}$ 是一个多面体（Dantzig 1956）。随后的重要进展包括：凸分析的丰富理论（Rockafellar 1970），以及线性优化问题的多项式时间求解方法（Khachiyan 1979，Karmarkar 1984）和广泛类别的非线性凸优化问题（Nesterov 和 Nemirovskii 1994）。

经典的优化问题是确定性的，即假设所有问题数据都是已知且确定的。然而，实践中遇到的大多数决策问题都依赖于那些受到测量误差影响的参数，或者这些参数仅在决策必须做出并且被确认之后才能揭示。将不确定性影响的决策问题简单地建模为确定性优化问题的一种方法是用它们的期望值或适当的点预测值替代所有不确定的参数。然而，早在很久以前就有人发现并记录了这样一个事实：那些将不确定的参数替换为其均值的决策者往往会陷入“平均误差”的陷阱（Savage, Scholtes 和 Zweidler 2006，Savage 2012）。为了考虑那些偏离均值的不确定性实现，Beale（1955）和Dantzig（1955）独立提出了以下形式的随机规划：

$$
\inf _{x \in \mathcal{X}} \mathbb{E}_{\mathbb{P}}[\ell(x, Z)],
$$

这些方法明确地将不确定的问题参数 $Z$ 建模为一个由概率分布 $\mathbb{P}$ 支配的随机向量，并且在此基础上寻找在期望下表现最好的决策（或根据某些风险度量进行优化）。此后，随机规划已发展成为一个成熟的领域（Birge 和 Louveaux 2011，Shapiro, Dentcheva 和 Ruszczyński 2009），并为机器学习中的经验风险最小化原则提供了理论基础（Bishop 2006，Hastie, Tibshirani 和 Friedman 2009）。

尽管随机规划在理论和实践中取得了成功，但它至少存在两个缺点。首先，假设概率分布 $\mathbb{P}$ 是完全已知的，在许多实际情况下是不现实的，且随机规划对这种分布的误设非常敏感。不同领域对这一效应的描述有：优化者的诅咒（Smith 和 Winkler 2006），优化的误差最大化效应（Michaud 1989，DeMiguel 和 Nogales 2009），优化偏差（Shapiro 2003）或过拟合（Bishop 2006，Hastie 等 2009）。其次，评估固定决策的期望损失需要计算多维积分，而这对于非常简单的损失函数和分布来说也已经是一个非常复杂的问题。因此，随机规划受到维度灾难的困扰，即它们的计算复杂度通常在随机向量 $Z$ 的维度上呈指数增长。为了缓解这两个缺点，Soyster（1973）提出将不确定性影响的决策问题建模为鲁棒优化问题，形式为：

$$
\inf _{x \in \mathcal{X}} \sup _{z \in \mathcal{Z}} \ell(x, z).
$$

鲁棒优化用基于集合的描述代替了不确定问题参数的概率描述，寻找在最坏情况下的参数实现 $z$ 下表现最好的决策。经过长时间的忽视，Soyster（1973）的思想在90年代末得到了重新审视并大大扩展，Kouvelis 和 Yu（1997），El Ghaoui、Oustry 和 Lebret（1998），El Ghaoui 和 Lebret（1998a,b），BenTal 和 Nemirovski（1999b, 1998, 1999a），Bertsimas 和 Sim（2004）等人进一步推进了该领域。有关鲁棒优化文献的综述，请参见 Ben-Tal、El Ghaoui 和 Nemirovski（2009），Rustem 和 Howe（2009）和 Bertsimas 和 den Hertog（2022）。我们指出，类似的思想在鲁棒稳定性（Horn 和 Johnson 1985，Doyle、Glover、Khargonekar 和 Francis 1989，Green 和 Limebeer 1995）和鲁棒控制（Zames 1966，Khalil 1996，Zhou、Doyle 和 Glover 1996）领域中也得到了独立发展，这些领域研究系统在参数变化下是否能够保持稳定，或者在参数变化下设计出具有良好性能的系统。有关鲁棒稳定性和控制的教材介绍，请参见 Zhou 和 Doyle（1999）和 Dullerud 和 Paganini（2001）。Hansen 和 Sargent（2008）将鲁棒控制技术应用于受模型不确定性影响的经济问题，设计在多种可能的模型误设下都能表现良好的政策。

尽管鲁棒优化减少了随机规划中的信息和计算负担，但它对模糊集内所有参数实现的平等对待及其对最坏情境的专注，可能使得它在实际应用中过于保守。这些问题促使研究人员研究分布鲁棒优化问题，形式为：

$$
\inf _{x \in \mathcal{X}} \sup _{\mathbb{P} \in \mathcal{P}} \mathbb{E}_{\mathbb{P}}[\ell(x, Z)],
$$

该模型将不确定的参数$Z$建模为一个随机向量，其由一个模糊集$\mathcal{P}$中的某个分布$\mathbb{P}$支配，寻求在最坏分布$\mathbb{P} \in \mathcal{P}$下，其期望值表现最好的决策。分布鲁棒优化（DRO）因此将随机编程的分布视角与鲁棒优化的最坏情况聚焦结合在一起。Herbert E. Scarf通常被认为是这一方法的先驱，他在关于新闻供应商问题的研究中提出，未知的需求分布仅通过其均值和方差来表征（Scarf 1958）。随后，Dupačová（1966，1987，1994）和Shapiro与Kleywegt（2002）研究了那些模糊集规定了未知概率分布的支持集、某些低阶矩、独立模式或其他结构特性的问题。Ermoliev、Gaivoronski 和 Nedeva（1985）及 Gaivoronski（1991）为基于矩模糊集的DRO问题开发了早期的求解方法。现代DRO的出现通常归功于Bertsimas和Popescu（2002，2005）的工作，他们在部分分布信息下推导了概率不等式，并将其技术应用于期权定价问题，El Ghaoui、Oks和Oustry（2003）以及Calafiore和El Ghaoui（2006）的研究，后者研究了DRO问题，其中目标函数的某个分位数应当最小化，或一组受不确定性影响的约束应当以高概率满足，跨所有具有已知矩约束的概率分布，以及Delage和Ye（2010）的研究，后者研究了类似的DRO问题，目标为最坏情况下的期望值目标。

早期关于DRO的研究主要集中在矩模糊集上，这些模糊集包含所有满足有限多个矩约束的指定支持集$\mathcal{Z}$上的分布。与随机规划不同，具有矩模糊集的DRO问题有时会在随机向量$Z$的维度上展现出有利的扩展性。然而，截然不同的分布可能具有相同的矩。因此，矩模糊集总是包含广泛的分布，其中包括一些不太可能的分布，当有充足的历史数据时，这些不太可能的分布可以安全地被排除。这促使Ben-Tal、den Hertog、De Waegenaere、Melenberg和Rennen（2013）以及Wang、Glynn和Ye（2016）引入了模糊集，这些模糊集包含所有在某个参考分布的邻域内的分布（通常是从历史数据形成的经验分布）。这些邻域可以通过概率分布之间的差异函数来定义，例如$\phi$-散度（Csiszar 1963）或Wasserstein距离（Villani 2008）。与矩模糊集不同，基于差异的模糊集具有可调的大小参数（例如，半径），因此可以缩小为仅包含参考分布的单例。如果参考分布收敛到未知的真实分布，并且随着更多历史数据的增加，大小参数趋近于0，则DRO问题最终会简化为在真实分布下的经典随机规划。早期关于基于差异的模糊集的研究假定$Z$是一个离散随机向量，具有有限的支持集$\mathcal{Z}$。基于差异的DRO问题扩展到一般的（可能是连续）随机向量的工作由Mohajerin Esfahani 和 Kuhn（2018），Zhao 和 Guan（2018），Blanchet 和 Murthy（2019），Zhang、Yang 和 Gao（2024b）以及 Gao 和 Kleywegt（2023）完成，他们使用最优传输差异构建了模糊集。我们请参阅Kuhn、Mohajerin Esfahani、Nguyen 和 Shafieezadeh-Abadeh（2019）以及Rahimian 和 Mehrotra（2022）以获取DRO文献的先前综述。

“分布鲁棒性”一词最早源于鲁棒统计。Huber（1981）首次提出该术语，用以描述在数据中存在异常值时做出鲁棒决策的方法。这个概念扩展了Box（1953，1979）的早期工作，后者研究了在基础分布偏离正态分布时的鲁棒性，正态分布是许多统计模型中的常见假设。为了应对异常值带来的挑战，统计学家们发展了几种污染模型，每个模型提供了不同的解决数据异常的途径。Huber污染模型由Huber（1964，1968）提出，并由Hampel（1968，1971）进一步发展，假设观察到的数据是从真实分布和任意污染分布的混合中抽取的。邻域污染模型通过统计距离定义偏离真实分布的情况，例如总变差（Donoho 和 Liu 1988）或Wasserstein距离（Zhu、Jiao 和 Steinhardt 2022a，Liu 和 Loh 2023）。最近，数据依赖的自适应污染模型允许将部分观察到的数据点替换为从任意分布中抽取的点（Diakonikolas、Kamath、Kane、Li、Moitra 和 Stewart 2019，Zhu 等 2022a）。有趣的是，DRO模型的乐观对偶模型，优化的是模糊集中的最佳分布（而非最坏分布），恢复了许多来自鲁棒统计的估计量（Blanchet、Li、Lin 和 Zhang 2024b，Jiang 和 Xie 2024）。关于算法鲁棒统计的最新进展，请参阅Diakonikolas 和 Kane（2023）。

鲁棒优化和分布鲁棒优化在机器学习中找到了广泛的应用。例如，机器学习文献中的流行正则化器已知具有鲁棒性解释，这为正则化在实践中的强大经验表现提供了理论洞察（Xu，Caramanis 和 Mannor 2009，Shafieezadeh-Abadeh、Kuhn 和 Mohajerin Esfahani 2019，Li、Lin、Blanchet 和 Nguyen 2022，Gao、Chen 和 Kleywegt 2024）。同样，DRO模型的乐观对偶模型，优化的是模糊集中的最佳（而非最坏）分布，产生了上置信区间算法，这些算法在赌博和强化学习文献中非常普遍（Blanchet 等 2024b，Jiang 和 Xie 2024）。DRO也与对抗训练有关，对抗训练旨在通过在对抗性样本的视角下训练模型，从而提高机器学习模型的泛化性能（Goodfellow、Shlens 和 Szegedy 2015）。对抗样本是现有数据点的扰动，旨在误导模型做出不正确的预测。


DRO 与扩展的随机（动态）编程之间也存在深刻的联系，这些扩展通过用一致的风险度量代替期望值。与期望值相似，风险度量将随机变量映射到扩展实数上。与期望值不同，期望值是风险中立的，因为它对正负结果赋予相同的权重，而风险度量通常赋予负面结果更大的权重，从而考虑到决策者经常表现出的风险厌恶。Artzner、Delbaen、Eber 和 Heath (1999) 以及 Delbaen (2002) 表明，满足一致性公理以及 Fatou 性质的风险度量可以等效地表示为在特定分布集上的最坏情况期望值。换句话说，优化最坏情况期望值（如DRO中所做的）和优化一致风险度量之间有直接的联系。在金融数学文献中，针对一类非线性期望——所谓的 $G$-期望，已经发展了类似的表示定理，这些期望基于反向随机微分方程的解（Peng 1997, 2007a,b, 2019）。Peng (2023) 显示，次线性 $G$-期望等价于在特定分布族上的最坏情况期望值，从而在 $G$-期望理论和DRO之间架起了桥梁。

从哲学上讲，DRO 与模糊厌恶原则相关，在这种原则下，个体更喜欢已知风险而不是未知风险，即使未知风险可能带来更高的回报。在经济学文献中，风险性结果的概率已知与模糊性结果的概率（部分）未知之间的区别可以追溯到至少凯恩斯（1921）和奈特（1921）。模糊厌恶的概念通过Ellsberg悖论（Ellsberg 1961）广泛传播，这是一种思想实验，在该实验中，人们被要求在一个已知分布的彩色球（例如50个红球和50个蓝球）的 urn 和一个具有未知分布的相同彩色球的 urn 之间做出选择（即红蓝球的比例未知）。尽管可能有相等或更好的赔率，许多人仍然更愿意在已知分布的 urn 上下注，即他们表现出模糊厌恶。Ellsberg悖论挑战了经典的期望效用理论，它导致了如最大最小期望效用理论（Gilboa 和 Schmeidler 1989）等扩展，作为DRO的理论基础。模糊厌恶随后在金融市场（Epstein 和 Miao 2003，Bossaerts、Ghirardato、Guarnaschelli 和 Zame 2010）、保险市场（Cabantous 2007）、个人决策（Dimmock、Kouwenberg 和 Wakker 2016）、宏观经济政策（Hansen 和 Sargent 2010）、拍卖（Salo 和 Weber 1995）以及信任博弈（Li、Turmunkh 和 Wakker 2019b）中被发现。

同时，也有大量医学和神经科学证据支持模糊厌恶的存在。Hsu、Bhatt、Adolphs、Tranel 和 Camerer（2005）发现，当个体面临模糊性时，大脑中的关键情绪处理中心——杏仁体的活动比面对已知概率的情境时更为活跃，这表明杏仁体在驱动模糊厌恶中的作用。Krain、Wilson、Arbuckle、Castellanos 和 Milham（2006）的荟萃分析突出了前额叶皮层的作用，该区域负责高级认知控制、理性决策和情绪调节，用于处理模糊性。此外，Wu、Sun、Camilleri、Eickhoff 和 Yu（2021）进行的荟萃分析表明，风险和模糊的处理都依赖于前岛叶。风险处理还会激活背内侧前额叶皮层和腹侧纹状体，而模糊处理则特定地涉及背外侧前额叶皮层、下顶叶和右前岛叶。这支持了这样一个观点：当个体面对模糊决策与风险决策时，涉及的是不同的神经机制。遗传因素可能影响个体对模糊厌恶的倾向。He、Xue、Chen、Lu、Dong、Lei、Ding、Li、Li、Chen、Li、Moyzis 和 Bechara（2010）将某些遗传多态性与个体在风险和模糊决策下的表现联系起来。在另一项研究中，Buckert、Schwieren、Kudielka 和 Fiebach（2014）研究了荷尔蒙变化（例如，与压力和焦虑相关的皮质醇水平升高）如何影响风险和模糊决策。这些发现共同表明，风险和模糊的感知不仅仅是认知现象，还受到大脑结构、遗传和荷尔蒙因素的影响，这些因素塑造了个体在模糊性决策中的差异。最后，我们提到Hartley 和 Somerville（2015）以及 Blankenstein、Crone、van den Bos 和 van Duijvenvoorde（2016），他们研究了模糊厌恶如何在儿童、青少年和成人之间有所不同，Hayden、Heilbronner 和 Platt（2010）观察到，当恒河猕猴在选择风险和模糊游戏时，表现出模糊厌恶。

本综述的其余部分结构如下。我们的分析的一个重要部分是研究最坏期望值 $\sup _{\mathrm{P} \in \mathcal{P}} \mathbb{E}_{\mathrm{P}}[\ell(x, Z)]$，这是DRO问题（1.2）的目标函数。评估这个表达式通常需要求解一个半无穷优化问题，这个问题有无限多个变量来表征概率分布 $P$，并且受到模糊集 $\mathcal{P}$ 施加的有限个约束。我们称这个问题为“自然的子问题”，它是区分DRO问题（1.2）与确定性、随机和鲁棒优化问题的关键特征。第2和第3节回顾了常见的模糊集 $\mathcal{P}$ 及其拓扑性质，特别关注自然子问题在什么条件下能够达到最优值。第4和第5节为自然子问题开发了对偶性理论，使我们能够上界或等价地重新表述最坏期望值，使用一个半无穷优化问题，通过有限多个对偶决策变量，施加无限多个约束。这个对偶性框架为第6节自然子问题的解析解奠定了基础，后者依赖于构建原始和对偶可行解，二者产生相同的目标值，从而享有强对偶性。第7和第8节利用相同的对偶性理论，开发了自然子问题以及整个DRO问题（1.2）的等价重构和保守逼近。第9节展示了对偶性理论如何为自然子问题和完整DRO问题提供数值求解技术。最后，第10节回顾了不同模糊集所享有的统计保障。

由于篇幅限制，本综述在选择涵盖的主题时面临艰难的取舍。我们决定聚焦于最常用的模糊集，并简要回顾其他可能的选择，例如边际模糊集、具有结构约束的模糊集（包括对称性和单峰性）、Sinkhorn模糊集或条件相对熵模糊集。同样，我们没有涵盖分布优先优化和决策随机化等重要但略显复杂的话题。最后，我们专注于单阶段问题，其中不确定性在作出此时此刻的决策 $x \in \mathcal{X}$ 后完全解决；两阶段和多阶段DRO问题，其中不确定性随时间展开并且可以进行补救决策，已由 Delage 和 Iancu（2015）以及 Yanikoglu、Gorissen 和 den Hertog（2019）进行了综述。





## 1.1 符号

本文中考虑的所有向量空间都定义在实数域上。为了简洁起见，我们简单地称它们为“向量空间”而不是“实向量空间”。我们使用 $\overline{\mathbb{R}} = \mathbb{R} \cup \{-\infty, \infty\}$ 来表示扩展实数。函数 $f: \mathbb{R}^d \rightarrow \overline{\mathbb{R}}$ 的有效域定义为 $\operatorname{dom}(f) = \{z \in \mathbb{R}^d : f(z) < \infty\}$，而 $f$ 的上图定义为 $\operatorname{epi}(f) = \{(z, \alpha) \in \mathbb{R}^d \times \mathbb{R}: f(z) \leq \alpha\}$。我们称 $f$ 为适当的，当且仅当 $\operatorname{dom}(f) \neq \emptyset$ 且对于所有 $z \in \mathbb{R}^d$，$f(z) > -\infty$。$f$ 的凸共轭是函数 $f^*: \mathbb{R}^d \rightarrow \overline{\mathbb{R}}$，通过 $f^*(y) = \sup_{z \in \mathbb{R}^d} y^{\top} z - f(z)$ 定义。一个凸函数 $f$ 被称为闭合的，如果它是适当的并且下半连续，或者它恒等于 $+\infty$ 或 $-\infty$。可以证明，$f$ 是闭合的当且仅当它与其双共轭 $f^{**}$ 一致，即与 $f^*$ 的共轭一致。如果 $f$ 是适当的、凸的且下半连续的，那么它的退化函数 $f^{\infty}: \mathbb{R}^d \rightarrow \mathbb{R}$ 通过以下公式定义：$f^{\infty}(z) = \lim_{\alpha \to \infty} \alpha^{-1}(f(z_0 + \alpha z) - f(z_0))$，其中 $z_0$ 是 $\operatorname{dom}(f)$ 中的任意一点（Rockafellar 1970，第8.5定理）。$f$ 的透视图是函数 $f^\pi: \mathbb{R}^d \times \mathbb{R} \rightarrow \overline{\mathbb{R}}$，通过以下公式定义：$f^\pi(z, t) = t f(z / t)$（当 $t > 0$ 时），$f^\pi(z, t) = f^{\infty}(z)$（当 $t = 0$ 时），$f^\pi(z, t) = \infty$（当 $t < 0$ 时）。可以证明，$f^\pi$ 是适当的、凸的且下半连续的（Rockafellar 1970，第67页）。在没有混淆的风险时，我们偶尔使用 $t f(z / t)$ 来表示 $f^\pi(z, t)$，即使 $t = 0$。集合 $\mathcal{Z} \subseteq \mathbb{R}^d$ 的指示函数 $\delta_{\mathcal{Z}}: \mathbb{R}^d \rightarrow \mathbb{R}$ 定义为：如果 $z \in \mathcal{Z}$，则 $\delta_{\mathcal{Z}}(z) = 0$；如果 $z \notin \mathcal{Z}$，则 $\delta_{\mathcal{Z}}(z) = \infty$。$\delta_{\mathcal{Z}}$ 的共轭 $\delta_{\mathcal{Z}}^*$ 被称为 $\mathcal{Z}$ 的支持函数。它满足 $\delta_{\mathcal{Z}}^*(y) = \sup_{z \in \mathcal{Z}} y^{\top} z$。随机对象用大写字母表示（例如，$Z$），其实现用小写字母表示（例如，$z$）。对于任意闭集 $\mathcal{Z} \subseteq \mathbb{R}^d$，我们使用 $\mathcal{M}(\mathcal{Z})$ 来表示所有有限签名的Borel测度空间，而 $\mathcal{M}_+(\mathcal{Z})$ 表示 $\mathcal{M}(\mathcal{Z})$ 中所有（非负的）Borel测度的凸锥，$\mathcal{P}(\mathcal{Z})$ 表示 $\mathcal{M}_+(\mathcal{Z})$ 中所有概率分布的凸集。关于 $P \in \mathcal{P}(\mathcal{Z})$ 的期望算子定义为：$\mathbb{E}_{P}[f(Z)] = \int_{\mathcal{Z}} f(z) \mathbb{dP}(z)$，对于任何 Borel 函数 $f: \mathcal{Z} \rightarrow \overline{\mathbb{R}}$。如果 $f$ 的正负部分的积分都为 $\infty$，则我们“敌对”地定义 $\mathbb{E}_{P}[f(Z)]$。即，如果积分出现在最小化（最大化）问题的目标函数中，则我们设定 $\mathbb{E}_{P}[f(Z)] = \infty$（或 $-\infty$）。Dirac 概率分布将单位概率赋给 $z \in \mathcal{Z}$，记作 $\delta_z$。Dirac 分布 $\delta_z$ 不应与单点集合 $\{z\}$ 的指示函数 $\delta_{\{z\}}$ 混淆。对于任何 $\mathcal{P} \in \mathcal{P}(\mathcal{Z})$ 和任何 Borel 可测变换 $f: \mathcal{Z} \rightarrow \mathcal{Z}^{\prime}$，其中 $\mathcal{Z} \subseteq \mathbb{R}^d$ 和 $\mathcal{Z}^{\prime} \subseteq \mathbb{R}^{d^{\prime}}$，我们用 $\mathbb{P} \circ f^{-1}$ 表示在 $f$ 下的推前分布。因此，如果 $Z$ 是一个由 $P$ 支配的在 $\mathcal{Z}$ 上的随机向量，那么 $f(Z)$ 就是一个由 $\mathbb{P} \circ f^{-1}$ 支配的在 $\mathcal{Z}^{\prime}$ 上的随机向量。一个集合 $\mathcal{Z} \subseteq \mathbb{R}^d$ 的闭包、内点和相对内点分别用 $\mathrm{cl}(\mathcal{Z})$、$\operatorname{int}(\mathcal{Z})$ 和 $\operatorname{rint}(\mathcal{Z})$ 表示。我们使用 $\mathbb{R}_+^d$ 和 $\mathbb{R}_{++}^d$ 来表示 $\mathbb{R}^d$ 中的非负正交体及其内点。此外，我们使用 $\mathbb{S}^d$ 来表示所有对称矩阵的空间，$\mathbb{R}^{d \times d}$ 中的对称矩阵。正半定矩阵的锥体用 $\mathbb{S}_+^d$ 表示，而 $\mathbb{S}_{++}^d$ 表示其内点，即所有正定矩阵的集合。逻辑命题 $\mathbb{1}_{\mathcal{E}}$ 的真值为1，当且仅当 $\mathcal{E}$ 为真，否则为0。所有自然数集合 $\{1, 2, 3, \ldots\}$ 用 $\mathbb{N}$ 表示，$[n] = \{1, \ldots, n\}$ 表示所有小于等于 $n$ 的整数的集合。

## 2. 模糊集合

模糊集合 $\mathcal{P}$ 是一个概率分布族，定义在一个公共的可测空间上。本文中，我们假设 $\mathcal{P} \subseteq \mathcal{P}(\mathcal{Z})$，其中 $\mathcal{P}(\mathcal{Z})$ 表示所有 Borel 概率分布的全集，这些分布定义在闭集 $\mathcal{Z} \subseteq \mathbb{R}^d$ 上。本节回顾了几种常见的模糊集合类别。对于每个类别，我们首先给出正式定义并提供历史背景信息。随后，我们举出重要的模糊集合实例，并突出它们的应用。




### 2.1. 矩量模糊集合

矩量模糊集合是一个满足有限个（广义的）矩量条件的概率分布族。形式上，它可以表示为：

$$
\mathcal{P}=\left\{\mathrm{P} \in \mathcal{P}(\mathcal{Z}): \mathrm{E}_{\mathrm{P}}[f(\mathrm{Z})] \in \mathcal{F}\right\},
$$

其中 $f: \mathcal{Z} \rightarrow \mathbb{R}^m$ 是一个 Borel 可测矩量函数，$\mathcal{F} \subseteq \mathbb{R}^m$ 是一个不确定性集合。根据定义，矩量模糊集合（2.1）包含所有支持在 $\mathcal{Z}$ 上的概率分布 $P$，其广义矩量 $\mathbb{E}_{\mathrm{P}}[f(Z)]$ 是定义良好的，并且属于不确定性集合 $\mathcal{F}$。类型为（2.1）的模糊集合首次由 Isii $(1960, 1962)$ 和 Karlin 与 Studden (1966) 研究，用于建立广义切比雪夫不等式的精确度。以下小节回顾了矩量模糊集合的常见实例。


#### 2.1.1. 仅支持模糊集合

仅支持模糊集合包含所有支持在 $\mathcal{Z} \subseteq \mathbb{R}^d$ 上的概率分布，即 $\mathcal{P} = \mathcal{P}(\mathcal{Z})$。它可以视为（2.1）的一种实例，设 $f(z) = 1$ 且 $\mathcal{F} = \{1\}$。任何具有模糊集合 $\mathcal{P}(\mathcal{Z})$ 的 DRO 问题，本质上都等同于一个经典的鲁棒优化问题，其不确定性集合为 $\mathcal{Z}$，即：

$$
\inf _{x \in \mathcal{X}} \sup _{P \in \mathcal{P}(\mathcal{Z})} \mathrm{B}_{\mathrm{P}}[\ell(x, Z)] = \inf _{x \in \mathcal{X}} \sup _{z \in \mathcal{Z}} \ell(x, z) .
$$

有关鲁棒优化的理论和应用的综合回顾，请参见 (Ben-Tal 和 Nemirovski 1998, 1999a, 2000, 2002, Bertsimas 和 Sim 2004, Ben-Tal 等人 2009, Bertsimas, Brown 和 Caramanis 2011, Ben-Tal, den Hertog 和 Vial 2015a, Bertsimas 和 den Hertog 2022)。

如果不确定集合 $\mathcal{Z}$ 覆盖了某个分布 $\mathbb{P}$ 的总概率质量的 $1-\varepsilon$ 的一部分，那么最坏情况损失 $\sup _{z \in \mathcal{Z}} \ell(x, z)$ 保证超过在 $\mathbb{P}$ 下 $\ell(x, Z)$ 的 $(1-\varepsilon)$-分位数。这可以通过利用 $\mathbb{P}$ 的先验结构信息或统计数据来实现。例如，如果 $\mathbb{P}(Z \in \mathcal{Z}) \geq 1-\varepsilon$（确定成立），则可以通过构造一个合适大小的半空间和椭球体的交集来满足这一条件，且如果 $Z$ 在 $\mathbb{P}$ 下具有独立的、对称的、单峰的和/或次高斯分量（Bertsimas and Sim 2004, Janak, Lin and Floudas 2007, BenTal et al. 2009, Li, Ding and Floudas 2011, Bertsimas, den Hertog and Pauphilet 2021）。或者，如果 $\mathcal{Z}$ 是通过使用统计假设检验（Postek, den Hertog and Melenberg 2016, Bertsimas, Gupta and Kallus 2018b,a）、分位数估计（Hong, Huang and Lam 2021）或基于学习的方法（Han, Shang and Huang 2021, Goerigk and Kurtz 2023, Wang, Becker, Van Parys and Stellato 2023）从 $\mathbb{P}$ 的独立样本中构建的，则可能高置信度成立。

### 2.1.2. Markov 模糊集合

Markov 不等式为一个非负单变量随机变量 $Z$ 提供了一个上界，该随机变量的均值为 $\mu \geq 0$，且其超过一个正阈值 $\tau>0$ 的概率。正式地，它表明对于模糊集合 $\mathcal{P}=\left\{\mathbb{P} \in \mathcal{P}\left(\mathbb{R}_{+}\right): \mathbb{E}_{\mathbb{P}}[Z]=\mu\right\}$ 中的每个可能的 $Z$ 的概率分布，$\mathbb{P}(Z \geq \tau) \leq \mu / \tau$。如果 $\mu \leq \tau$，那么 Markov 不等式是紧的，即存在一个概率分布 $\mathbb{P}^{\star} \in \mathcal{P}$，使得该不等式成立为等式。实际上，分布 $\mathbb{P}^{\star}=(1-\mu / \tau) \delta_0 + \mu / \tau \delta_\tau$，其中 $\delta_z$ 是 Dirac 分布，它将点质量放置在 $z \in \mathbb{R}$ 上，属于 $\mathcal{P}$ 并满足 $\mathbb{P}(Z \geq \tau) = \mu / \tau$。这些洞察表明 $\sup _{\mathbb{P} \in \mathcal{P}} \mathbb{P}(Z \geq \tau) = \mu / \tau$，并且当 $\mu \leq \tau$ 时，最大值由 $\mathbb{P}^{\star}$ 达到。因此，Markov 上界可以被解释为一个 DRO 问题的最优值。因此，通常将 $\mathcal{P}$ 称为 Markov 模糊集合。更一般地，我们将与闭合支持集 $\mathcal{Z} \subseteq \mathbb{R}^d$ 和均值向量 $\mu \in \mathbb{R}^d$ 对应的 Markov 模糊集合定义为形式为

$$
\mathcal{P}=\left\{\mathrm{P} \in \mathcal{P}(\mathcal{Z}): \mathbb{E}_{\mathrm{P}}[Z]=\mu\right\} .
$$

因此，Markov 模糊集合（2.2）包含所有支持在 $\mathcal{Z}$ 上的分布，这些分布具有相同的均值向量 $\mu$。然而，这些分布可能具有截然不同的形状和高阶矩。Markov 模糊集合上的最坏情况期望有时被用作在随机规划中对期望成本函数的有效计算上界。如果成本函数在不确定问题参数中是凹的，则这些最坏情况期望与 Jensen 不等式紧密相关（Jensen 1906）；详见第 6.1 节。另一方面，如果成本函数是凸的，并且 $\mathcal{Z}$ 是一个多面体，则这些最坏情况期望与 Edmundson-Madansky 不等式相关（Edmundson 1956，Madansky 1959）；详见第 6.2 节。



### 2.1.3. Chebyshev 模糊集合

Chebyshev 不等式提供了一个上界，说明一个具有有限均值 $\mu \in \mathbb{R}$ 和方差 $\sigma^2>0$ 的单变量随机变量 $Z$ 偏离其均值超过 $k>0$ 个标准差的概率。正式地，它表明对于模糊集合 $\mathcal{P}=\left\{\mathbb{P} \in \mathcal{P}(\mathbb{R}): \mathbb{E}_{\mathbb{P}}[Z]=\mu, \mathbb{E}_{\mathbb{P}}\left[Z^2\right]=\sigma^2+\mu^2\right\}$ 中的每个可能的 $Z$ 的概率分布，$\mathbb{P}(|Z-\mu| \geq k \sigma) \leq 1 / k^2$。如果 $k \geq 1$，则 Chebyshev 不等式是紧的。实际上，可以很容易验证分布

$$
\mathbb{P}^{\star}=\frac{1}{2 k^2} \delta_{\mu-k \sigma}+\left(1-\frac{1}{k^2}\right) \delta_\mu+\frac{1}{2 k^2} \delta_{\mu+k \sigma}
$$

是 $\mathcal{P}$ 的一个元素，并且满足 $\mathbb{P}(|Z-\mu| \geq k \sigma)=1 / k^2$。这些洞察表明，Chebyshev 上界可以被解释为一个 DRO 问题的最优值。因此，通常将 $\mathcal{P}$ 称为 Chebyshev 模糊集合。更一般地，我们将与闭合支持集 $\mathcal{Z} \subseteq \mathbb{R}^d$、均值向量 $\mu \in \mathbb{R}^d$ 和二阶矩阵 $M \in \mathbb{S}_{+}^d, M \geq \mu \mu^{\top}$ 对应的 Chebyshev 模糊集合定义为

$$
\mathcal{P}=\left\{\mathrm{P} \in \mathcal{P}(\mathcal{Z}): \mathrm{E}_{\mathrm{P}}[Z]=\mu, \mathbb{E}_{\mathrm{P}}\left[Z Z^{\top}\right]=M\right\}
$$

因此，Chebyshev 模糊集合（2.3）包含所有支持在 $\mathcal{Z}$ 上的分布，这些分布具有相同的均值向量 $\mu$ 和二阶矩阵 $M$（因此也具有相同的协方差矩阵 $\Sigma=M-\mu \mu^{\top} \in \mathbb{S}_{+}^d$）。然而，这些分布可能具有截然不同的形状和高阶矩。

Chebyshev 模糊集合（2.3）捕捉了与多变量 Chebyshev 不等式相关的分布信息（Lal 1955，Marshall 和 Olkin 1960，Tong 1980，Rujeerapaiboon, Kuhn 和 Wiesemann 2018）。在运筹学中，Chebyshev 模糊集合自 Scarf（1958）关于分布鲁棒新闻供应商的开创性工作以来被广泛使用，该工作通常被认为是 DRO 的第一篇论文。从那时起，许多与 Chebyshev 模糊集合相关的 DRO 模型在新闻供应商和投资组合选择问题的背景下涌现。这些模型涉及各种不同的决策标准，如期望值（Gallego 和 Moon 1993，Natarajan 和 Linyi 2007，Popescu 2007），风险价值（El Ghaoui 等 2003，Xu, Caramanis 和 Mannor 2012b，Zymler, Kuhn 和 Rustem 2013a,b，Rujeerapaiboon, Kuhn 和 Wiesemann 2016，Yang 和 Xu 2016，Zhang, Jiang 和 Shen 2018），条件风险价值（Natarajan, Sim 和 Uichanco 2010，Chen, He 和 Zhang 2011，Zymler 等 2013b，Hanasusanto, Kuhn, Wallace 和 Zymler 2015a），谱风险度量（Li 2018）和失真风险度量（Cai, Li 和 Mao 2023，Pesenti, Wang 和 Wang 2024），以及最大最小遗憾标准（Yue, Chen 和 Wang 2006，Perakis 和 Roels 2008）。2008)。除此之外，切比雪夫模糊集在期权和股票定价（Bertsimas 和 Popescu 2002）、统计学和机器学习（Lanckriet, El Ghaoui, Bhattacharyya 和 Jordan 2001, 2002, Strohmann 和 Grudic 2002, Huang, Yang, King, Lyu 和 Chan 2004, Bhattacharyya 2004, Farnia 和 Tse 2016, Nguyen, Shafieezadeh-Abadeh, Yue, Kuhn 和 Wiesemann 2019, Rontsis, Osborne 和 Goulart 2020）、随机规划（Birge 和 Wets 1986, Dulá 和 Murthy 1992, Dokov 和 Morton 2005, Bertsimas, Doan, Natarajan 和 Teo 2010, Natarajan, Teo 和 Zheng 2011）、控制（Van Parys, Kuhn, Goulart 和 Morari 2015, Yang 2018, Xin 和 Goldberg 2021, 2022）、电力系统运行（Xie 和 Ahmed 2017, Zhao 和 Jiang 2017）、复杂网络分析（Van Leeuwaarden 和 Stegehuis 2021, Brugman, Van Leeuwaarden 和 Stegehuis 2022）、排队系统（van Eekelen, Hanasusanto, Hasenbein 和 van Leeuwaarden 2023）、医疗保健（Mak, Rong 和 Zhang 2015, Shehadeh, Cohn 和 Jiang 2020）、极端事件分析（Lam 和 Mottet 2017）等方面都得到了广泛应用。


2.1.4. 切比雪夫模糊集与不确定的矩

在已知 $\mathbb{P}$ 的一阶和二阶矩时，使用切比雪夫模糊集是合适的，但所有更高阶的矩是未知的。然而，在实际中，即使是已知的一阶和二阶矩也从来不能绝对确定。相反，它们必须从统计数据中估计，因此会受到估计误差的影响。这促使 El Ghaoui 等人（2003）引入了具有不确定矩的切比雪夫模糊集，可以表示为：

$$
\mathcal{P}=\left\{\mathbb{P} \in \mathcal{P}(\mathcal{Z}):\left(\mathrm{E}_{\mathrm{P}}[Z], \mathrm{B}_{\mathrm{P}}\left[Z Z^{\top}\right]\right) \in \mathcal{F}\right\} .
$$

这里，$\mathcal{F} \subseteq \mathbb{R}^d \times \mathbb{S}_{+}^d$ 是一个凸集，用于捕捉矩的不确定性。显然，$\mathcal{P}$ 可以表示为一系列切比雪夫模糊集的并集，即：

$$
\mathcal{P}=\bigcup_{(\mu, M) \in \mathcal{F}}\left\{\mathrm{P} \in \mathcal{P}(\mathcal{Z}): \mathrm{E}_{\mathrm{P}}[Z]=\mu, \mathbb{E}_{\mathrm{P}}\left[Z Z^{\top}\right]=M\right\} .
$$

注意，具有不确定矩的切比雪夫模糊集涵盖了支持集模糊集、马尔可夫模糊集和切比雪夫模糊集作为特例。它们通过设置 $\mathcal{F}=\mathbb{R}^d \times \mathbb{S}_{+}^d, \mathcal{F}=\{\mu\} \times \mathbb{S}_{+}^d$ 和 $\mathcal{F}=\{\mu\} \times\{M\}$ 分别恢复。

El Ghaoui 等人（2003）通过以下盒子捕捉矩的不确定性：

$$
\mathcal{F}=\left\{(\mu, M) \in \mathbb{R}^d \times \mathbb{S}_{+}^d: \underline{\mu} \leq \mu \leq \bar{\mu}, \underline{M} \leq M \leq \bar{M}\right\}
$$

该框由矩的界限参数化，其中 $\underline{\mu}, \bar{\mu} \in \mathbb{R}^d$ 和 $\underline{M}, \bar{M} \in \mathbb{S}_{+}^d$。

给定对未知均值向量和协方差矩阵的噪声估计 $\hat{\mu}$ 和 $\hat{\Sigma}$，Delage 和 Ye（2010）提出了以下模糊集：

$$
\mathcal{P}=\left\{\mathbb{P} \in \mathcal{P}(\mathcal{Z}): \begin{array}{l}
\left(\mathbb{E}_{\mathrm{P}}[Z]-\hat{\mu}\right)^{\top} \hat{\Sigma}^{-1}\left(\mathbb{E}_{\mathrm{P}}[Z]-\hat{\mu}\right) \leq \gamma_1 \\
\mathbb{E}_{\mathrm{P}}\left[(Z-\hat{\mu})(Z-\hat{\mu})^{\top}\right] \leq \gamma_2 \hat{\Sigma}
\end{array}\right\} .
$$

通过构造，$\mathcal{P}$ 包含所有分布在 $\mathcal{Z}$ 上，其一阶矩位于以 $\hat{\mu}$ 为中心的椭球内，二阶矩（相对于 $\hat{\mu}$）位于以 $\gamma_2 \hat{\Sigma}$ 为顶点的半正定锥内。一个基本的计算揭示了：

$$
\mathbb{E}_{\mathrm{P}}\left[(Z-\hat{\mu})(Z-\hat{\mu})^{\top}\right]=\mathrm{E}_{\mathrm{P}}\left[Z Z^{\top}\right]-\mathbb{B}_{\mathrm{P}}[Z] \hat{\mu}^{\top}-\hat{\mu} \mathrm{E}_{\mathrm{P}}[Z]^{\top}+\hat{\mu} \hat{\mu}^{\top}
$$
